In [1]:
from Classifications import Classifications
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import compute_class_weight
import glob
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
classifications = Classifications()
model_h_path = "../ModelData.h"
scaler_h_path = "../Scaler.h"
data_csvs = glob.glob("indoors_sensor_data*.csv")
np_data = "data_processing/data/stroke_peak_data.npz"
batch_size = 128
num_epochs = 512
patience = 256
seed = 42
rng = np.random.default_rng(seed=seed)
test_split = 0.2
val_split = 0.2

In [3]:
model = Sequential([
    layers.InputLayer((classifications.num_steps, classifications.num_features), dtype=tf.float16),

    layers.Conv1D(32, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.Dropout(0.1),

    layers.Conv1D(64, 3, padding="same"),
    layers.BatchNormalization(),
    layers.Activation("relu"),
    layers.MaxPooling1D(pool_size=2),
    layers.Dropout(0.1),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.4),
    layers.Dense(classifications.num_classes, activation="softmax"),
])

model.summary()

2025-04-30 11:34:50.820019: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-04-30 11:34:50.820039: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-04-30 11:34:50.820044: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
I0000 00:00:1746027290.820059 24906373 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1746027290.820078 24906373 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 64, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 64, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 64, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 13)             │           845 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,573 (1.06 MB)

 Trainable params: 278,381 (1.06 MB)

 Non-trainable params: 192 (768.00 B)

In [4]:
sample_input = tf.random.uniform(shape=(1, classifications.num_steps, classifications.num_features), dtype=tf.float16, seed=seed)
logits = model.predict(sample_input)

2025-04-30 11:34:50.974035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


In [5]:
prediction = tf.argmax(logits, axis=1)[0]
print(prediction)

tf.Tensor(7, shape=(), dtype=int64)


In [6]:
prediction_string = ' '.join(reversed(classifications.classes[prediction]))
print(f"Prediction: {prediction_string}")

Prediction: slice backhand groundstroke


In [7]:
df = pd.concat([pd.read_csv(data_csv) for data_csv in data_csvs], ignore_index=True)

# Peak detection
peaks, _ = find_peaks(
    np.sum(df[["ax", "ay", "az"]].values ** 2, axis=1),
    height=classifications.squared_acceleration_threshold,
    distance=classifications.num_steps,
)

sensor_columns = ["ax", "ay", "az", "gx", "gy", "gz"]

X = []
y = []

for peak in peaks:
    start_idx = peak - classifications.steps_before_peak
    end_idx = peak + classifications.steps_after_peak
    if start_idx < 0 or end_idx >= len(df):
        continue

    shot_df = df.loc[start_idx:end_idx]
    assert len(shot_df) == classifications.num_steps

    shot_data = shot_df[sensor_columns].values
    stroke = df.loc[peak, "stroke"].lower()
    side = df.loc[peak, "side"].lower()
    spin = df.loc[peak, "spin"].lower()
    label_key = (stroke, side, spin)

    if label_key in classifications.classes:
        label = classifications.class_to_idx[label_key]
        X.append(shot_data)
        y.append(label)

X = np.array(X).astype(np.float16)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, random_state=seed, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_split, random_state=seed, stratify=y_train)

scaler = StandardScaler()
scaler.fit(X_train.reshape(-1, X_train.shape[-1]))

with open(scaler_h_path, "w") as f:
    f.write("#ifndef _SCALER_H_\n#define _SCALER_H_\n\n#include \"SensorData.h\"\n\n")
    f.write("constexpr float mean_[NUM_FEATURES] = {")
    f.write(",".join(f"{x}" for x in scaler.mean_))
    f.write("};\n")
    f.write("constexpr float scale_[NUM_FEATURES] = {")
    f.write(",".join(f"{x}" for x in scaler.scale_))
    f.write("};\n")
    f.write("\n#endif\n")

def transform_data(X, scaler):
    num_samples, num_steps, num_features = X.shape
    X_flat = X.reshape(-1, num_features)
    X_scaled_flat = scaler.transform(X_flat)
    return X_scaled_flat.reshape(num_samples, num_steps, num_features)

X_train = transform_data(X_train, scaler)
X_val = transform_data(X_val, scaler)
X_test = transform_data(X_test, scaler)

def random_rotate_sample_tf(sample, label):
    def random_rotation_matrix():
        u1 = tf.random.uniform([], 0, 1)
        u2 = tf.random.uniform([], 0, 1)
        u3 = tf.random.uniform([], 0, 1)

        q1 = tf.sqrt(1 - u1) * tf.sin(2 * np.pi * u2)
        q2 = tf.sqrt(1 - u1) * tf.cos(2 * np.pi * u2)
        q3 = tf.sqrt(u1) * tf.sin(2 * np.pi * u3)
        q4 = tf.sqrt(u1) * tf.cos(2 * np.pi * u3)

        x, y, z, w = q1, q2, q3, q4
        rot = tf.stack([
            [1 - 2*y*y - 2*z*z,     2*x*y - 2*z*w,     2*x*z + 2*y*w],
            [2*x*y + 2*z*w,     1 - 2*x*x - 2*z*z,     2*y*z - 2*x*w],
            [2*x*z - 2*y*w,         2*y*z + 2*x*w, 1 - 2*x*x - 2*y*y]
        ])
        return rot

    R = random_rotation_matrix()
    R = tf.cast(R, sample.dtype)  # Match rotation matrix type to sample

    accel = sample[:, :3]
    gyro = sample[:, 3:]

    rotated_accel = tf.linalg.matmul(accel, R, transpose_b=True)
    rotated_gyro = tf.linalg.matmul(gyro, R, transpose_b=True)

    rotated_sample = tf.concat([rotated_accel, rotated_gyro], axis=1)
    return rotated_sample, label

train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .shuffle(len(X_train), seed=seed)
    .map(random_rotate_sample_tf, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

In [8]:
#import matplotlib.pyplot as plt
#
#sample, label = next(iter(train_ds))
#sample = sample[0]
#label = label[0]
#
#plot_width = 6
#x_ticks = np.arange(len(sample))
#
## Raw data plot
#plots = (
#    { "label": "ax", "title": "Acceleration X", "color": "r" },
#    { "label": "ay", "title": "Acceleration Y", "color": "g" },
#    { "label": "az", "title": "Acceleration Z", "color": "b" },
#    { "label": "gx", "title": "Gyroscope X", "color": "r" },
#    { "label": "gy", "title": "Gyroscope Y", "color": "g" },
#    { "label": "gz", "title": "Gyroscope Z", "color": "b" },
#)
#
#fig, axes = plt.subplots(len(plots), 1, figsize=(plot_width, 3 * len(plots)))
#for i, (ax, plot) in enumerate(zip(axes, plots)):
#    ax.plot(x_ticks, sample[:, i], label=plot["title"], color=plot["color"])
#    ax.set_xlabel("Milliseconds")
#    ax.set_ylabel(plot["title"])
#    ax.set_xticks(x_ticks[::3])
#    ax.ticklabel_format(style='plain')
#    ax.grid(True)
#
#plt.tight_layout()
#plt.show()

In [9]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

checkpoint_callback = ModelCheckpoint(
    filepath="best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
)

early_stopping = EarlyStopping(
    monitor="val_accuracy",
    patience=patience,
    restore_best_weights=True,
    verbose=1,
)

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i: class_weights[np.where(np.unique(y_train) == i)[0][0]] if i in np.unique(y_train) else 0 for i in range(classifications.num_classes)}

history = model.fit(
    train_ds,
    epochs=num_epochs,
    callbacks=[checkpoint_callback, early_stopping],
    validation_data=val_ds,
    class_weight=class_weights,
    verbose=1,
)

Epoch 1/512
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.0494 - loss: 5.7160
Epoch 1: val_accuracy improved from -inf to 0.13415, saving model to best_model.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 175ms/step - accuracy: 0.0517 - loss: 5.7119 - val_accuracy: 0.1341 - val_loss: 2.5092
Epoch 2/512
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0781 - loss: 5.3620
Epoch 2: val_accuracy did not improve from 0.13415
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0795 - loss: 5.7898 - val_accuracy: 0.1341 - val_loss: 2.4740
Epoch 3/512
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0547 - loss: 7.3884
Epoch 3: val_accuracy did not improve from 0.13415
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0898 - loss: 7.4209 - val_accuracy: 0.1098 - val_loss: 2.3953
Epoch 4/512
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0625 - loss: 8.0524
Epoch 4: val_accuracy did not improve from 0.13415
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0958 - loss: 8.3488 - val_accu

In [10]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6765 - loss: 0.9483
Test Loss: 0.9483287930488586
Test Accuracy: 0.6764705777168274


In [12]:
#model = tf.keras.models.load_model(f"data_processing/data/best_fold_5.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset():
    for input_value in tf.data.Dataset.from_tensor_slices(X_train.astype(np.float16)).batch(1).take(100):
        yield [input_value]
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type  = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/1y/nzsqhm41529c176c8k4bw7s80000gn/T/tmprp6mlrsn/assets


INFO:tensorflow:Assets written to: /var/folders/1y/nzsqhm41529c176c8k4bw7s80000gn/T/tmprp6mlrsn/assets


Saved artifact at '/var/folders/1y/nzsqhm41529c176c8k4bw7s80000gn/T/tmprp6mlrsn'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 6), dtype=tf.float16, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 13), dtype=tf.float32, name=None)
Captures:
  13093414160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093415696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093414544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093415312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093413968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093416272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093416848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093418960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093415888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093418768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13093418192: Te

/Users/jozef/Documents/personal/Spring 2025/ML/Design Project/SmartSportsSensor/.venv/lib/python3.12/site-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1746027333.567528 24906373 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1746027333.567540 24906373 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-30 11:35:33.567695: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/1y/nzsqhm41529c176c8k4bw7s80000gn/T/tmprp6mlrsn
2025-04-30 11:35:33.568118: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-30 11:35:33.568121: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/1y/nzsqhm41529c176c8k4bw7s80000gn/T/tmprp6mlrsn
I0000 00:00:1746027333.572228 24906373 mlir_graph_optimization_pass.cc:425] MLIR V1

In [13]:
print(f"Model size: {len(tflite_model)} bytes")

with open(model_h_path, "w") as f:
    f.write("#ifndef _MODELDATA_H_\n#define _MODELDATA_H_\n")
    f.write("const unsigned char model[] = {")
    f.write(",".join(f"0x{b:02x}" for b in tflite_model))
    f.write("};\n#endif\n")

Model size: 292760 bytes
